## NLP Foundations
* Mostly Solved: Spam detection, Named Entity Recognition, POS 
* Good Progress: Sentiment Analysis, Parsing, Information Extraction
* Hard: Q&A, Summarization, Paraphrase

**Ambiguity is pervasive:** Phrase Structure Parse - noun/verb phrases. Depending how a sentence is interpreted, we could have different phrase structures. "Federal raises interest rates." Raises could be the verb. Or interest could be the verb.
* Difficulty comes from non-standard english, correct segmentation, idioms (dark horse), trick entity names (when movie names start and end).
* Use probabilistic models from language data.
* Regular Language: disjunction [wW]oodchuck and ranges can involve [A-Za-z] to cover all letters lowercase/uppercase. We can have negation in disjunction. [^Ss] or [^A-Z]. 
    * Pipe: disjunction meaning "or".
    * ? matches optional previous character.
    * * is a kleene star or matches 0 or more of a previous character.
    * + matches to 1 or more previous character.
    * . matches with anything (only 1 character).
    * Anchor: ^[A-Z] is for the start (capital letters at start).
        * \\.\$ means string should end with a period.
    * Processing of modifying regexes involves minimizing false positives or negatives. We add more to regex to qualify it.
* Lexer - takes sequences and cuts it to pieces of tokens (tokenizer).
* Tokenization - found that vocabulary increases more than O($\sqrt{N}$) for N being the number of tokens.
    * Type - unique words in vocabulary.
    * Tokenization can have issues regarding apostrophes - removes those and results in standalone letters such as "d".
    * To achieve it on text, you can use piping: $\verb|tr -sc "A-Za-z" "\n" | sort | uniq -c | sort -r -n | less |$. c is the complement so we replace all characters that aren't alphabetical with a newline.
    * Tokenization issues with other languages is the absence of spaces in chinese/japanese. L'ensemble should be broken up as two tokens or remain as one token? These are some challenges.
    * To tackle this, we apply a baseline tokenization algorithm: maximum-matching. 
        * (1) Begin with a wordlist in chinese and a string. Direct a pointer to start of string and find longest string matching starting at pointer. Move the pointer over and repeat. Thetabledownthere - probabilistic segmentation algorithm.
* Normalize/Stem Words - index/query terms should be matched. For information retrieval, we need to ensure U.S.A and USA are mapped to the same.
    * Alternative asymmetric expansion: Enter: window, Search: windows.
    * Case folding - generally reduce all letters to lowercase. But may not if capital in middle of sentence. SAIL vs sail or Fed vs. fed.
    * Lemmatization - reduce variants to the base form. am, are, is -> be. This is significant in machine translation - quiero (I want) and quieres (You want) have the same lemma of querer.
    * Stemming - crude chopping off of affixes. Stemming is simplified version of lemmatization.
    * Topic of looking at parts of words: Morpheme (meaningful unit of language that cannot be further divided) - stems (meaning bearing) and affixes (pieces that adhere to stems - grammatical functions)
    * Porter's Algorithm (replace rules) - sses -> ss (chops off es). Rules are implemented in order. (*vowel*)ing -> none, (*vowel*)ed -> none. For longer stems, al -> none. Corpus linguistics in Linux.
    * Sentence Segmentation - ., ? are ambiguous since . can end an abbreviation or numbers like 4.3. One can use a decision tree to determine whether . is the end of a sentence or not.
        * Features in decision tree classifier can be more advanced - case of word after "." - probability of word with "." occurring at end of sentence.
* Similarity between 2 strings: use embedding and use cosine similarity. Or minimum edit distance - editing done through substitution, deletion, insertion. Two words & alignment.
    * In **levenstein distance**, substitution has double cost of deletion and insertion.
    * In machine translation, we want to see how well translation perform.
    * For named entity extraction and entity coreference, if IBM and IBM Inc. are the same. Number of words different to improve entity identification. 
    * Define initial state, operators, goal state, and path cost - cannot search whole space (shortest path to each revisited state).
    * Edit distance can be done using dynamic programming - combining solutions to subproblems.
    * Minimum edit distance may not be enough - we need to align the characters of two strings to each other. To do this, we keep a backtrace. The backtrace would include multiple pointers to the possible entries that could have lead to a specific cell. This reveals a path starting from the last indices to [0][0] 
    * **Weighted Edit Distance** - spell correction (some letters more likely than others). a is more likely to be confused with e,i,o. Modified algorithm to add cost of each deletion/insertion/substitution costs.
    * **Minimum Edit Distance in Computational Biology** - maximizes similarity/score instead of minimizing weight in NLP - uses needle-wunsch algorithm - d being deletion/insertion while substitution has custom costs. Origin at upper left for comp bio. Initialization is at -i*d, -j*d, where d is the cost of deleting/inserting something. When updating deletion/insertion, you subtract d. For substitution, you add substitution cost.
        * To accomodate for number of gaps at start/end, we don't penalize gaps. **Overlap detection variant** - initialization so we don't have costs to delete or insert everything. We initialize costs where either index is 0 to be 0, where we terminate from the max of the maxes from last column/row. 
        * **Smith-Waterman Algorithm (Local alignment problem)** - we would want to optimize alignment - similar maximized. So we ignore badly aligned regions. For iteration, we consider maximums of 0, deletion/insertion reduced by d, and additional substitution cost. We trace back in the matrix to identify the optimal alignments leading to maximized similarity - +1 for match, -1 for del/ins/sub.
* Measure how good a translation algorithm is by probability of sequence of words. Probabilistic language models are used.
    * Joint probability of words in a sentence: 
    * Count and divide all such sentence, but just too many. The markov assumption is that .

In [1]:
import re
import torch 

In [69]:
# levenstein distance and backtracing alignment
def min_edit_distance(s1,s2):
    table = [[[0,None] for j in range(len(s1)+1)] for i in range(len(s2)+1)]
    for i in range(len(table)):
        table[i][0][0] = i
    for j in range(len(table[0])):
        table[0][j][0] = j
    for i in range(1,len(table)):
        for j in range(1,len(table[i])):
            alignment_directions = [(table[i-1][j][0]+1,"DELETE"),(table[i][j-1][0]+1,"INSERT")]
            if s2[i-1] != s1[j-1]:
                alignment_directions.append((table[i-1][j-1][0]+2,"SUBSTITUTE"))
            else:
                alignment_directions.append((table[i-1][j-1][0],"SAME CHAR"))
            min_direction = [float("inf"),None]
            for direction in alignment_directions:
                if min_direction[0] >= direction[0]:
                    min_direction = direction
            table[i][j] = min_direction
    row, col = len(s2),len(s1)
    alignment_steps = []
    for line in table:
        for val,direction in line:
            print(val," ",end="")
        print("\n")
    while row >= 0 and col >= 0:
        direction = table[row][col][1]
        alignment_steps = [direction + f" for {s1[:(col+1)]} and {s2[:(row+1)]}"] + alignment_steps
        if direction in ["SUBSTITUTE","SAME CHAR"]:
            row -= 1
            col -= 1
        elif direction == "DELETE":
            row -= 1
        elif direction == "INSERT":
            col -= 1
        if table[row][col][1] is None:
            assert col == 0 or row == 0, "None should only be in entries along edge of rows/cols."
            if col == 0:
                which_str = s2
                chars_added = s2[:(row+1)]
                alignment_steps = [f"INSERT characters from {which_str}: {chars_added}"] + alignment_steps
            else:
                which_str = s1
                chars_added = s1[:(col+1)]
                alignment_steps = [f"INSERT characters from {which_str}: {chars_added}"] + alignment_steps
            break
    return table[len(s2)][len(s1)], alignment_steps

print(min_edit_distance("intention","execution"))

0  1  2  3  4  5  6  7  8  9  

1  2  3  4  3  4  5  6  7  8  

2  3  4  5  4  5  6  7  8  9  

3  4  5  6  5  6  7  8  9  10  

4  5  6  7  6  7  8  9  10  11  

5  6  7  8  7  8  9  10  11  12  

6  7  8  7  8  9  8  9  10  11  

7  6  7  8  9  10  9  8  9  10  

8  7  8  9  10  11  10  9  8  9  

9  8  7  8  9  10  11  10  9  8  

((8, 'SAME CHAR'), ['INSERT characters from intention: in', 'SUBSTITUTE for int and ex', 'SUBSTITUTE for inte and exe', 'SAME CHAR for inten and exec', 'DELETE for inten and execu', 'SUBSTITUTE for intent and execut', 'SAME CHAR for intenti and executi', 'SAME CHAR for intentio and executio', 'SAME CHAR for intention and execution', 'SAME CHAR for intention and execution'])


In [70]:
# Needleman-Wunsch Algorithm - we maximize the negated values of edits.
# Problem: Don't want to penalize/add costs for unlimited gaps at ends.

def create_genetic_code_table(seq1,seq2,substitution,d):
    table = [[[0,None] for j in range(len(seq1)+1)] for i in range(len(seq2)+1)]
    for i in range(len(table)):
        table[i][0][0] = -i*d
    for j in range(len(table[0])):
        table[0][j][0] = -j*d
    for i in range(1,len(table)):
        for j in range(1,len(table[i])):
            alignment_directions = [(table[i-1][j][0]-d,"DELETE"),(table[i][j-1][0]-d,"INSERT")]
            alignment_directions.append((table[i-1][j-1][0]+substitution(seq1[j],seq2[i]),"SUBSTITUTE"))
            min_direction = [float("inf"),None]
            for direction in alignment_directions:
                if min_direction[0] >= direction[0]:
                    min_direction = direction
            table[i][j] = min_direction
            
            table[i][j] = max(max(table[i-1][j]-d,table[i][j-1]-d),
                             table[i-1][j-1]+substitution_cost_matrix[seq1[i]][seq2[j]])
    return table

def nwunsch(seq1,seq2,substitution_cost_matrix,d=1):
    table = create_genetic_code_table(seq1,seq2,substitution,d)
    # termination of maximum distance
    return table[len(seq2)-1][len(seq1)-1]
# No cost from starting out with nothing - variant involves 2 overlapping reads from a sequencing project.
def overlap(seq1,seq2,substitution,d=1):
    table = create_genetic_code_table(seq1,seq2,substitution,d)
    # backtracing begins from the maximum column or row.
    last_col = (0,float("-inf"))
    last_row = (0,float("-inf"))
    for i in range(len(table)):
        entry = (i,table[i][len(seq2)-1])
        if entry[1] > max_cost[1]:
            last_col = entry
            
    for j in range(len(table[0])):
        entry = (i,table[j][len(seq1)-1])
        if entry[1] > max_cost[1]:
            last_row = entry
    if last_col[0] > last_row[0]:
        start = (last_col[0],len(seq2)-1)
    else:
        start = (len(seq1)-1,last_row[0])
    
    row,col = start[0],start[1]
    while row >= 0 and col >= 0:
        direction = table[row][col][1]
        alignment_steps = [direction + f" for {s1[:(col+1)]} and {s2[:(row+1)]}"] + alignment_steps
        if direction in ["SUBSTITUTE","SAME CHAR"]:
            row -= 1
            col -= 1
        elif direction == "DELETE":
            row -= 1
        elif direction == "INSERT":
            col -= 1
        if table[row][col][1] is None:
            assert col == 0 or row == 0, "None should only be in entries along edge of rows/cols."
            
            break
#overlap("AGTCCTGACCGT","TTCATAGTCGGCCAG", substitution=[[],[],[]],d=1)

TypeError: overlap() missing 1 required positional argument: 'substitution'

In [ ]:
# Longest Common Substring - similarity is maximized.
# Allow for 
# Smith-Waterman Algorithm
def smith_waterman():
    pass

## NLP Understanding
* Task of Relation Extraction - extraction of triplets (relation_name, first_item, second_item)
    * The first item is related to the second in some way.
    * Ex: (founders, SpaceX, Elon_Musk)
    * Automating this can quickly expand knowledge base.
    * 60's/70's - Query "X is the founder of Y". Consider variants of the query, but may not consider "Elon Musk, co-founder of SpaceX..."
    * Statistics/Data - feature representation to allow for better generalization. Constraint: **Manually annotate** each sentence. Negative examples would not have a relation.
        * **Innovation** - distant supervision. Just use external resource as source of truth. Search corpus and identify sentence with two items, assuming they have the relation specified in knowledge base. 100 times as much data.
            * **Limitations** - makes assumption of co-occurrence. Injects noise to dataset (noisy labels). Only able to extend knowledge, but cannot be used to create new knowledge base.
    * Polysemy/Synonymy

## Information Retrieval
* Boolean Retrieval